# Práctica 4

**Nombre:** Esparza Duran Kenia *Jaqueline*

**e-mail:** kenia.esparza5866@alumnos.udg.mx


## Instrucciones
**Requisitos mínimos:**
* Funciones que generen trayectorias tipo Brownian Motion (BM), Correlated Random
* Walk (CRW) y Lévy Flight (LF).
* Cada una de las funciones deberá tomar como parámetros el numero de pasos, la velocidad y posición inicial. Además la funciones para CRW  y LF deberán tomar como parámetro el coeficiente para la distribución Cauchy.
* Por último, la función para LF también deberá aceptar como parámetro el exponente Lévy (alpha).

**El dashboard degerá contener al menos los siguientes elementos:**
* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a generar y análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante (segun sea el caso) para ajustar los parámetros de las trayectorias. Estos widgets deberán mostrarse de manera dinámica, es decir, solo deberan mostrarse los parametros relevantes para cada tipo de trayectoria.
* Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcular de la trayectoria bajo análisis.
Revisión de la práctica:
* El notebook con la práctica se entregará en un repositorio en GitHub. Será necesario demostrar por medio de commits el historial de versiones de su NoteBook.
* El estudiante deberá ser capaz de explicar su código y de corregir errores introducidos a este.

## MODULES

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import Layout, interactive
from scipy.stats import cauchy
from IPython.display import display


Estas funciones generan trayectorias en 3D con diferentes modelos de movimiento:

* generate_BM: Modelo de Movimiento Browniano.
* generate_CRW: Caminata Aleatoria Continua.
* generate_LF: Vuelo de Lév

In [16]:
# Definiciones de funciones para generar trayectorias
def generate_BM(steps, velocity, initial_position):
    deltas = np.random.normal(loc=0, scale=velocity, size=(steps, 3))
    trajectory = np.cumsum(deltas, axis=0) + initial_position
    return trajectory

def generate_CRW(steps, velocity, initial_position, cauchy_coef):
    angles = cauchy.rvs(loc=0, scale=cauchy_coef, size=steps)
    x = np.cos(angles) * velocity
    y = np.sin(angles) * velocity
    z = np.random.normal(loc=0, scale=velocity, size=steps)  # Z uses normal distribution
    deltas = np.vstack([x, y, z]).T
    trajectory = np.cumsum(deltas, axis=0) + initial_position
    return trajectory

def generate_LF(steps, velocity, initial_position, cauchy_coef, alpha):
    distances = np.random.pareto(alpha, size=steps) + velocity
    angles = cauchy.rvs(loc=0, scale=cauchy_coef, size=steps)
    x = np.cos(angles) * distances
    y = np.sin(angles) * distances
    z = np.random.normal(loc=0, scale=velocity, size=steps)  # Z uses normal distribution
    deltas = np.vstack([x, y, z]).T
    trajectory = np.cumsum(deltas, axis=0) + initial_position
    return trajectory

Estas funciones calculan métricas sobre las trayectorias generadas:

* path_length(trajectory): Calcula la longitud total de la trayectoria como la suma de las distancias entre puntos sucesivos.

* mean_squared_displacement(trajectory): Calcula el desplazamiento cuadrático medio de la trayectoria, que es la media de las magnitudes cuadradas de los desplazamientos desde el punto inicial.

In [17]:
# Definiciones de funciones para calcular métricas
def path_length(trajectory):
    distances = np.sqrt(np.sum(np.diff(trajectory, axis=0)**2, axis=1))
    total_length = np.sum(distances)
    return total_length

def mean_squared_displacement(trajectory):
    displacement = trajectory - trajectory[0]
    msd = np.mean(np.sum(displacement**2, axis=1))
    return msd


Estas funciones son para visualizar trayectorias en 3D y métricas lineales asociadas:

* plot_trajectory(trajectory): Visualiza una trayectoria tridimensional en un gráfico 3D.

* plot_trajectory_and_metric(trajectory, metric_values, metric_name): Muestra la trayectoria en 3D en un panel y la métrica seleccionada en el otro panel.

In [ ]:
# Función para visualizar la trayectoria en 3D
def plot_trajectory(trajectory):
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(trajectory[:,0], trajectory[:,1], trajectory[:,2])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()
# Función modificada para visualizar la trayectoria y la métrica linealmente
def plot_trajectory_and_metric(trajectory, metric_values, metric_name):
    plt.figure(figsize=(15, 6))

    # Gráfico de trayectoria en 3D
    ax1 = plt.subplot(1, 2, 1, projection='3d')
    ax1.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2])
    ax1.set_title('3D Trajectory')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')

    # Gráfico lineal de la métrica
    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(metric_values, label=metric_name)
    ax2.set_title(f'{metric_name} Over Time')
    ax2.set_xlabel('Step')
    ax2.set_ylabel(metric_name)
    ax2.legend()

    plt.tight_layout()
    plt.show()

Este bloque de código define una interfaz interactiva para visualizar trayectorias y métricas en tiempo real:

* interactive_ui(): Función principal que crea la interfaz interactiva.
Utiliza widgets de ipywidgets para permitir al usuario seleccionar el tipo de trayectoria, el número de pasos, la velocidad inicial, coeficientes relevantes y la métrica a visualizar.
* La función update_plot() es llamada cada vez que cambia un widget. Calcula la trayectoria y la métrica correspondiente según las selecciones del usuario.
Se utiliza la función plot_trajectory_and_metric() para mostrar la trayectoria y la métrica seleccionada en dos paneles diferentes.

In [18]:

# Interfaz interactiva para visualización en tiempo real
def interactive_ui():
    traj_type_selector = widgets.RadioButtons(options=['BM', 'CRW', 'LF'], description='Trajectory Type:', style={'description_width': 'initial'})
    steps_widget = widgets.IntSlider(min=10, max=1000, step=10, value=100, description='Steps:', style={'description_width': 'initial'})
    velocity_widget = widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0, description='Velocity:', style={'description_width': 'initial'})
    initial_pos_widget = widgets.FloatSlider(min=-10.0, max=10.0, step=0.1, value=0.0, description='Initial Position:', style={'description_width': 'initial'})
    cauchy_coef_widget = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='Cauchy Coef:', style={'description_width': 'initial'})
    alpha_widget = widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Levy Alpha:', style={'description_width': 'initial'})
    metric_selector = widgets.Dropdown(options=['Path Length', 'Mean Squared Displacement'], description='Metric:', style={'description_width': 'initial'})

    def update_plot(traj_type, steps, velocity, initial_pos, cauchy_coef, alpha, metric):
        initial_position = np.array([initial_pos] * 3)
        if traj_type == 'BM':
            trajectory = generate_BM(steps, velocity, initial_position)
        elif traj_type == 'CRW':
            trajectory = generate_CRW(steps, velocity, initial_position, cauchy_coef)
        elif traj_type == 'LF':
            trajectory = generate_LF(steps, velocity, initial_position, cauchy_coef, alpha)

        if metric == 'Path Length':
            metric_values = [path_length(trajectory[:i+1]) for i in range(1, steps+1)]
        else:  # Mean Squared Displacement
            metric_values = [mean_squared_displacement(trajectory[:i+1]) for i in range(1, steps+1)]

        plot_trajectory_and_metric(trajectory, metric_values, metric)

    ui = interactive(update_plot, traj_type=traj_type_selector, steps=steps_widget, velocity=velocity_widget,
                     initial_pos=initial_pos_widget, cauchy_coef=cauchy_coef_widget, alpha=alpha_widget, metric=metric_selector)
    display(ui)

interactive_ui()

interactive(children=(RadioButtons(description='Trajectory Type:', options=('BM', 'CRW', 'LF'), style=Descript…